In [124]:
import torch
import torch.nn as nn
import torchvision.transforms as T
import glob
from PIL import Image

In [138]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, train):
        self.x = []
        self.y = []
        self.transform = T.Compose([T.RandomAffine((0, 360)), T.ToTensor()])
        # self.transform = T.Compose([T.ToTensor()])

        # 加载数据
        label2id = {'ellipse': 0, 'rect': 1, 'triangle': 2}
        for label, id in label2id.items():
            path = glob.glob(f'data/{label}/*.jpg')
            max_lens = max(map(len, path))
            if train:
                path = list(filter(lambda s: len(s) == max_lens, path))
            else:
                path = list(filter(lambda s: len(s) != max_lens, path))

            self.x.extend(path)
            self.y.extend([id] * len(path))
        
        self.y = torch.tensor(self.y)


    def __getitem__(self, index):
        im = Image.open(self.x[index])
        x = self.transform(im)

        return x[:3,...], self.y[index]

    def __len__(self):
        return len(self.y)


train_dataset = Dataset(True)
test_dataset = Dataset(False)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, num_workers=2, shuffle=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, num_workers=2)

In [135]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Sequential(nn.Conv2d(3, 4, 5, padding=2, stride=2), nn.ReLU())
        self.conv2 = nn.Sequential(nn.Conv2d(4, 4, 3, padding=1), nn.ReLU())
        self.conv3 = nn.Sequential(nn.Conv2d(4, 8, 3, padding=1, stride=2), nn.ReLU())
        self.conv4 = nn.Sequential(nn.Conv2d(8, 8, 3, padding=1), nn.ReLU())
        self.conv5 = nn.Sequential(nn.Conv2d(8, 16, 3, padding=1, stride=2), nn.ReLU())
        self.conv6 = nn.Sequential(nn.Conv2d(16, 16, 3, padding=1), nn.ReLU())
        self.conv7 = nn.Sequential(nn.Conv2d(16, 8, 3, padding=1), nn.ReLU())

        self.linear = nn.Sequential(nn.Flatten(),
                                    nn.Linear(8*9*6, 128), nn.ReLU(), 
                                    nn.Linear(128, 3))

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.linear(x)

        return x

model = Model()
x = torch.rand((1, 3, 72, 48))
print(model(x).shape)

torch.Size([1, 3])


In [136]:
model = Model().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [140]:
for epoch in range(40):
    for step, (x, y) in enumerate(train_loader):
        x = x.to('cuda')
        y = y.to('cuda')
        pred_y = model(x)

        acc = y.eq(pred_y.max(dim=1)[1]).sum() / y.shape[0]
        loss = loss_fn(pred_y, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


        if epoch % 5 == 0 and step == 0:
            print(f"acc: {acc.item():>.3f}  softmax_loss: {loss.item():>.5f}   [{step:>3}/{len(train_loader):>3}]")

acc: 1.000  softmax_loss: 0.01318   [  0/163]
acc: 1.000  softmax_loss: 0.00572   [  0/163]
acc: 1.000  softmax_loss: 0.00085   [  0/163]
acc: 1.000  softmax_loss: 0.00049   [  0/163]
acc: 1.000  softmax_loss: 0.00021   [  0/163]
acc: 1.000  softmax_loss: 0.00023   [  0/163]
acc: 1.000  softmax_loss: 0.00513   [  0/163]
acc: 1.000  softmax_loss: 0.00006   [  0/163]


In [141]:
cnt = 0
true_cnt = 0
for x, y in test_loader:
    x = x.to('cuda')
    y = y.to('cuda')
    pred_y = model(x)

    cnt += len(y)
    true_cnt += y.eq(pred_y.max(dim=1)[1]).sum() 

    # print(y, pred_y.max(dim=1)[1])

print(true_cnt/cnt)

tensor(0.9600, device='cuda:0')
